In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from transformers import MarianMTModel, AutoTokenizer

print(f'pytorch version: {torch.__version__}')

num_gpu = torch.cuda.device_count()
print(f'{num_gpu} GPU available')

pytorch version: 2.2.1+cu121
0 GPU available


# Different generation strategies for MarianMTModel

In [ ]:
from transformers import MarianMTModel, AutoTokenizer
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
# src_texts = ["I am a small frog.", "Tom asked his teacher for advice."]
# tgt_texts = ["Ich bin ein kleiner Frosch.", "Tom bat seinen Lehrer um Rat."]
inputs = tokenizer("I am a small frog.", return_tensors="pt")
decoder_in = tokenizer("Ich bin", return_tensors="pt")

outputs = model(inputs.input_ids,decoder_input_ids=decoder_in.input_ids)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
# from transformers import MarianMTModel, AutoTokenizer
# model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
# # src_texts = ["I am a small frog.", "Tom asked his teacher for advice."]
# # tgt_texts = ["Ich bin ein kleiner Frosch.", "Tom bat seinen Lehrer um Rat."]
# inputs = tokenizer("I am a small frog.", return_tensors="pt")
# decoder_in = tokenizer("Ich bin", return_tensors="pt")

# outputs = model(inputs.input_ids,decoder_input_ids=decoder_in.input_ids)

## Below is slightly tweaked implementation of query_model that works with MarianMTModel

In [ ]:
def query_model_aggressive(transformer, text, tokenizer, device, max_length=25):

    inputs = tokenizer(text, return_tensors='pt').to(device)
    output_string = ''
    outputs = tokenizer(output_string, return_tensors='pt').to(device)
    outputs = outputs.input_ids
    model.to(device)

    i = 0
    next_token_id = tokenizer.cls_token_id
    while i < max_length and next_token_id != tokenizer.eos_token_id:
        i += 1

        probs = transformer(inputs.input_ids,decoder_input_ids=outputs).logits[0][-1]
        # print(probs)
        probs = F.softmax(probs, dim=-1)
        next_token_id = probs.argmax().unsqueeze(dim=0)
        outputs = torch.cat((outputs,next_token_id.unsqueeze(dim=0)),dim=1)
        # print(outputs)
        next_token = tokenizer.convert_ids_to_tokens(next_token_id)
        output_string = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0]))

    return output_string

In [ ]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
query_model_aggressive(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> </s>'

In [ ]:
query_model_aggressive(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> </s>'

In [ ]:
query_model_aggressive(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> </s>'

In [ ]:
query_model_aggressive(model, "I am a ittle child", tokenizer, "cpu", max_length=25)

'</s> </s>'

### Autoagressive strategy
Results are bad to say the least, lets try and implent strategy involving sampling.

### Samping strategy

### Exercise

Modify the `query_model` function so that each next token is chosen by sampling from the distribution of obtained softmax values, possibly considering only the top-k probabilities, for a fixed value of k (or, even better, by adding an optional parameter to the function).


In [ ]:
def query_model_sampling(transformer, text, tokenizer, device, max_length=25, k=20):

    inputs = tokenizer(text, return_tensors='pt').to(device)
    output_string = ''
    outputs = tokenizer(output_string, return_tensors='pt').to(device)
    outputs = outputs.input_ids
    model.to(device)

    i = 0
    next_token_id = tokenizer.cls_token_id
    while i < max_length and next_token_id != tokenizer.eos_token_id:
        i += 1

        probs = transformer(inputs.input_ids,decoder_input_ids=outputs).logits[0][-1]
        # print(probs)
        probs = F.softmax(probs, dim=-1)
        top_probs = torch.topk(probs,k)
        sampled_idx = torch.distributions.Categorical(top_probs.values).sample()
        next_token_id = top_probs.indices[sampled_idx]
        # print(next_token_id)
        outputs = torch.cat((outputs,next_token_id.unsqueeze(dim=0).unsqueeze(dim=0)),dim=1)
        # print(outputs)
        next_token = tokenizer.convert_ids_to_tokens(next_token_id.unsqueeze(dim=0))
        output_string = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0]))

    return output_string

In [ ]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> , ein Kind zur Unterstützung und zur Teilnahme an den Programmen „Custom a gen ib ti" und „<pad>'

In [ ]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> ,</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> ein.</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> ) zu er</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> auf sich)<pad> Ich bin ein wenig gereizt.</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> <pad> Ich bin ein bittle Kind</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> die der die der die des der</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> <pad> Ich bete einen jungen Mann als Kind ins Leben?</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> , auf der Stufe auf dem die EU ein sein will.</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> zu die,<pad> Ich bin ein itle kind,</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> die</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> )), auf.<pad> "<pad> Ich bin ein Itti-<pad> Ich bin ein kleiner</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=25)

'</s> <pad> Es sind nur einige der</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=30)

'</s> .<pad> Ich bin ein kleiner Jugendlicher</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=50)

'</s> .<pad> Ich bin ein kniekind<pad> Ich ich bin ein mutter</s>'

In [ ]:
query_model_sampling(model, "I am a ittle child", tokenizer, "cuda", max_length=50)

'</s> eine</s>'

Results are not great, but make much more sense.

### Beam search

### Exercise

Search on the Web a description of the «beam search» procedure for choosing the tokens to be added to the output when querying a transformer. Modify the `query_model` function accordingly.

**Note** This exercise is vaguely described on purpose. It represents a challenge!

In [ ]:
def query_model_beam_search(transformer, text, tokenizer, device, max_length=25, beams_n=2):

    inputs = tokenizer(text, return_tensors='pt').to(device)
    output_string = ''
    outputs = tokenizer(output_string, return_tensors='pt').to(device)
    outputs = outputs.input_ids
    model.to(device)
    tokens_n = tokenizer.vocab_size

    next_token_id = tokenizer.cls_token_id

    # at firts we run model on the input to get two most promising tokens
    probs = transformer(inputs.input_ids,decoder_input_ids=outputs).logits[0][-1]
    # print(probs)
    probs = F.softmax(probs, dim=-1)
    top_probs = torch.topk(probs,beams_n)
    next_token_ids = top_probs.indices

    # now we have beams_n candidates for next token and we want to investigate them further
    outputs = [outputs for _ in range(beams_n)]
    old_probs = probs

    for i in range(beams_n):
        outputs[i] = torch.cat((outputs[i],next_token_ids[i].unsqueeze(dim=0).unsqueeze(dim=0)),dim=1)

    # print(outputs)

    # prepare a tensor for picking right token index from all generated probabilites from all beams (will work also
    # when no. of beams will get smaller, when one beam gets to eos token) (?alternatively best_prob_idx - beams_n*vocab_size)

    all_token_ids = [[i for i in range(tokenizer.vocab_size)] for _ in range(beams_n)]
    all_token_ids = torch.tensor(all_token_ids, dtype=torch.int32, device=device).flatten()
    # print(all_token_ids[58100])
    seq_len = 0

    end_outputs = []
    end_probs = []

    while seq_len < max_length and beams_n>0:
        seq_len += 1

        all_probs = []

        for i in range(beams_n):
            # print("-----")
            probs = transformer(inputs.input_ids,decoder_input_ids=outputs[i]).logits[0][-1]
            probs = F.softmax(probs, dim=-1)
            # print(probs.size())
            probs = probs*old_probs[i]
            all_probs += list(probs)

        all_probs = torch.tensor(all_probs, device=device)

        top_probs = torch.topk(all_probs,beams_n)
        next_token_ids = all_token_ids[top_probs.indices]
        output_idx = top_probs.indices // tokens_n


        #now we have n_beams output + next_token pairs that gave best probability

        # print(top_probs)
        # print(next_token_ids)
        # print(output_idx)
        # print("-"*30)

        new_old_probs = []

        new_outputs = []
        for i in range(beams_n):
            if next_token_ids[i] == tokenizer.eos_token_id:
                beams_n -= 1
                end_outputs.append(torch.cat((outputs[output_idx[i]],next_token_ids[i].unsqueeze(dim=0).unsqueeze(dim=0)),dim=1))
                # print(old_probs[i])
                end_probs.append(top_probs.values[i])
            else:
                new_outputs.append(torch.cat((outputs[output_idx[i]],next_token_ids[i].unsqueeze(dim=0).unsqueeze(dim=0)),dim=1))
                new_old_probs.append(top_probs.values[i])

        outputs = new_outputs
        old_probs = new_old_probs
        # print(outputs)
        # print(old_probs)
        # print("="*30)

    end_outputs += outputs
    end_probs += old_probs

    best_seqs = []

    best_prob = end_probs[0]
    best_output = end_outputs[0]
    for i in range(len(end_outputs)):
        best_seqs.append(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(end_outputs[i][0])))
        if end_probs[i] >= best_prob:
            best_prob = end_probs[i]
            best_output = end_outputs[i]

    print("Following sequences with probabilities were found with beam search")

    for i in range(len(end_outputs)):
        print(f'seq: {best_seqs[i]}, prob: {end_probs[i]}')

    output_string = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(best_output[0]))

    print(f"Final output is: {output_string} with probability: {best_prob}")
    return output_string

In [ ]:
query_model_beam_search(model, "I am a ittle child", tokenizer, "cpu", max_length=10, beams_n=3)

Following sequences with probabilities were found with beam search
seq: </s> </s> </s>, prob: 0.009832926094532013
seq: </s> .</s>, prob: 0.006324343848973513
seq: </s> .<pad> Ich bin ein Kind der Itti</s>, prob: 2.2669061650049116e-07
Final output is: </s> </s> </s> with probability: 0.009832926094532013


'</s> </s> </s>'

In [ ]:
query_model_beam_search(model, "I am a ittle child", tokenizer, "cpu", max_length=15, beams_n=4)


Following sequences with probabilities were found with beam search
seq: </s> </s> </s>, prob: 0.009832926094532013
seq: </s> .</s>, prob: 0.006324343848973513
seq: </s> .<pad> Ich bin ein Kind</s>, prob: 6.603592191822827e-05
seq: </s> .<pad> Ich bin ein Kind der Itti</s>, prob: 2.2669061650049116e-07
Final output is: </s> </s> </s> with probability: 0.009832926094532013


'</s> </s> </s>'

In [ ]:
query_model_beam_search(model, "This dog would be in fact better as a cat", tokenizer, "cpu", max_length=20, beams_n=5)


Following sequences with probabilities were found with beam search
seq: </s> .</s>, prob: 0.004094363655894995
seq: </s> ,</s>, prob: 0.002659340389072895
seq: </s> , als</s>, prob: 0.0006782801356166601
seq: </s> .<pad> Dieser Hund wäre eigentlich besser als Katze</s>, prob: 2.5623665351304226e-05
seq: </s> .<pad> Dieser Hund wäre in der Tat besser als Katze</s>, prob: 2.8441043468774296e-05
Final output is: </s> .</s> with probability: 0.004094363655894995


'</s> .</s>'

In [ ]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

query_model_beam_search(model, "This dog would be in fact better as a cat", tokenizer, "cpu", max_length=20, beams_n=5)


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Following sequences with probabilities were found with beam search
seq: </s> .</s>, prob: 0.004094363655894995
seq: </s> ,</s>, prob: 0.002659340389072895
seq: </s> , als</s>, prob: 0.0006782801356166601
seq: </s> .<pad> Dieser Hund wäre eigentlich besser als Katze</s>, prob: 2.5623665351304226e-05
seq: </s> .<pad> Dieser Hund wäre in der Tat besser als Katze</s>, prob: 2.8441043468774296e-05
Final output is: </s> .</s> with probability: 0.004094363655894995


'</s> .</s>'

### Beam search results
Final result with beam search is bad, but if a end token has a high probability at the beggining it leads to less probability multiplication, and with each multiplication of a number \<1 with a number \<1 we get really small numbers, so the output somehow makes sense. When we consider results of all beams, we can find some pretty sensible outputs.

# Trying out analogical tasks for GPT2


In [ ]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id

sentence_list = ["I tasted an apple and it was"]
inputs = tokenizer(sentence_list, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Below is a `query_model` function adapted for using pretrained transformer (gpt2). As this type of transformer varies significantly from the one the function was created for, the modifications are significant

In [ ]:
def gpt2_query_model(transformer, text, tokenizer, device, max_length=25):

    inputs = tokenizer(text, return_tensors='pt').to(device)
    # print(inputs)
    input_ids = inputs.input_ids.to(device)
    output_string = ''
    model.to(device)

    i = 0
    next_token_id = tokenizer.cls_token_id
    while i < max_length and next_token_id != tokenizer.eos_token_id:
        i += 1

        model_out = transformer.generate(
                input_ids,
                max_new_tokens=1,
                return_dict_in_generate=True,
                output_scores=True,
                pad_token_id=tokenizer.eos_token_id
                )
        probs = F.softmax(model_out.scores[0], dim=-1)
        next_token_id = probs.argmax()
        input_ids = torch.cat((input_ids,next_token_id.unsqueeze(dim=0)[:,None]),dim=1)
        next_token = tokenizer.convert_ids_to_tokens(next_token_id.unsqueeze(dim=0))
        output_string += tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(next_token_id.unsqueeze(dim=0)))

    return output_string

text = "The sea tasted like a a a"
text_generated = gpt2_query_model(model, text, tokenizer, "cpu", max_length=40)
print(f'prompt: {text}')
print(f'generated: {text_generated}')
print("-------------")

prompt: The sea tasted like a a a
generated: ether, and the sea was like a sea of water.

The sea was like a sea of water.

The sea was like a sea of water.

The sea was like
-------------


In [ ]:
text = "This fish is possibly"
text_generated = gpt2_query_model(model, text, tokenizer, "cuda", max_length=40)
print(f'prompt: {text}')
print(f'generated: {text_generated}')
print("-------------")

prompt: This fish is possibly
generated:  the most popular fish in the world. It is a very popular fish in the United States. It is a very popular fish in the United States. It is a very popular fish in the United States
-------------


### Exercise

Modify the `query_model` function so that each next token is chosen by sampling from the distribution of obtained softmax values, possibly considering only the top-k probabilities, for a fixed value of k (or, even better, by adding an optional parameter to the function).


In [ ]:
def gpt2_query_model_sampling(transformer, text, tokenizer, device, max_length=25, k=10):
    inputs = tokenizer(text, return_tensors='pt').to(device)
    # print(inputs)
    input_ids = inputs.input_ids.to(device)
    output_string = ''
    model.to(device)

    i = 0
    next_token_id = tokenizer.cls_token_id
    while i < max_length and next_token_id != tokenizer.eos_token_id:
        i += 1

        model_out = transformer.generate(
                input_ids,
                max_new_tokens=1,
                return_dict_in_generate=True,
                output_scores=True,
                pad_token_id=tokenizer.eos_token_id
                )
        probs = F.softmax(model_out.scores[0], dim=-1)
        top_probs = torch.topk(probs,k)
        next_token_id = probs.argmax()
        sampled_idx = torch.distributions.Categorical(top_probs.values).sample()
        next_token_id = top_probs.indices[:,sampled_idx]
        input_ids = torch.cat((input_ids,next_token_id),dim=1)
        next_token = tokenizer.convert_ids_to_tokens(next_token_id.unsqueeze(dim=0))
        output_string += tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(next_token_id.unsqueeze(dim=0)))

    return output_string

text = "The sea tasted like a a a"
text_generated = gpt2_query_model_sampling(model, text, tokenizer, "cpu", max_length=40)
print(f'prompt: {text}')
print(f'generated: {text_generated}')
print("-------------")

prompt: The sea tasted like a a a
generated: peritif filled with the scent of the ocean, the sea smelled like the air. The sea was a world of water and the ocean was water.

And now, the sea was filled
-------------


In [ ]:
text = "This fish is possibly"
text_generated = gpt2_query_model(model, text, tokenizer, "cpu", max_length=40)
print(f'prompt: {text}')
print(f'generated: {text_generated}')
print("-------------")

prompt: This fish is possibly
generated:  the most popular fish in the world. It is a very popular fish in the United States. It is a very popular fish in the United States. It is a very popular fish in the United States
-------------


In [ ]:
text = "This fish is possibly"
text_generated = gpt2_query_model_sampling(model, text, tokenizer, "cpu", max_length=40)
print(f'prompt: {text}')
print(f'generated: {text_generated}')
print("-------------")

prompt: This fish is possibly
generated:  the most popular fish of the genus. It is also known from its distinctive color and distinctive head, and a long and slender tail.

The fish, called a koi, is the only
-------------


In [ ]:
text = "This fish is possibly"
text_generated = gpt2_query_model_sampling(model, text, tokenizer, "cpu", max_length=40)
print(f'prompt: {text}')
print(f'generated: {text_generated}')
print("-------------")

prompt: This fish is possibly
generated:  more dangerous than its name, which is probably true for most people. It is a great way to eat fish, and a great way to eat a fish that is not poisonous. The reason this fish
-------------
